In [ ]:
!pip3 install pycuda

     |████████████████████████████████| 1.6MB 4.8MB/s 
     |████████████████████████████████| 61kB 21.0MB/s 
     |████████████████████████████████| 471kB 23.1MB/s 
  Created wheel for pycuda: filename=pycuda-2019.1.2-cp36-cp36m-linux_x86_64.whl size=4536176 sha256=b79d9b2bb580b7f5627186a88e211c60fab548a3cc78ee238cba3958e6c2439a
  Stored in directory: /root/.cache/pip/wheels/a6/60/f0/b1c430c73d281ac3e46070480db50f7907364eb6f6d3188396
  Created wheel for pytools: filename=pytools-2019.1.1-py2.py3-none-any.whl size=58424 sha256=c2897372791321efe3316e3446fbc29d2ebe1032f34ef69d3e1c6023dd6cfc96
  Stored in directory: /root/.cache/pip/wheels/83/df/0b/75ac4572aaa93e3eba6a58472635d0fda907f5f4cf884a3a0c
  Created wheel for mako: filename=Mako-1.1.0-cp36-none-any.whl size=75363 sha256=8d5d659aeebd57d28f47652c982fc5c18a851692b49d437bd16978d482c012b6
  Stored in directory: /root/.cache/pip/wheels/98/32/7b/a291926643fc1d1e02593e0d9e247c5a866a366b8343b7aa27
Successfully built pycuda pytools mako


In [ ]:
import math
import numpy as np
import pycuda.gpuarray as gpuarray
from pycuda.compiler import SourceModule
import pycuda.autoinit

In [ ]:
module = SourceModule("""
texture<int, 1, cudaReadModeElementType> tex_1d;
__global__ void read_texture_1d(int nx, int *res){
    int x = threadIdx.x + blockDim.x * blockIdx.x;
    if (x < nx){
        res[x] = tex1Dfetch(tex_1d, x);
    }
}
""")

In [ ]:
read_tex_1d = module.get_function("read_texture_1d")

In [ ]:
num_components = np.int32(10)
arr = np.arange(num_components, dtype=np.int32)

In [ ]:
np.random.shuffle(arr)

In [ ]:
print(arr)

[4 5 7 1 9 0 8 2 3 6]


In [ ]:
arr_gpu = gpuarray.to_gpu(arr)
res_gpu = gpuarray.zeros(num_components, dtype=np.int32)

In [ ]:
tex_1d = module.get_texref("tex_1d")
arr_gpu.bind_to_texref_ext(tex_1d)

0.0

In [ ]:
thread_per_block = (256, 1, 1)
blocks_per_grid = (math.ceil(num_components / threads_per_block[0]), 1, 1)

In [ ]:
read_tex_1d(num_components, res_gpu, block=threads_per_block, grid=blocks_per_grid, texrefs=[tex_1d])

In [ ]:
res_gpu.get()

array([4, 5, 7, 1, 9, 0, 8, 2, 3, 6], dtype=int32)